<a href="https://colab.research.google.com/github/hagarbarakat/bigint-prime-check/blob/main/Algorithms_Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adding Google benchmark libraries**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

fatal: destination path 'benchmark' already exists and is not an empty directory.
fatal: destination path 'benchmark/googletest' already exists and is not an empty directory.
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Google Benchmark version: v1.9.4-61-g7b9e482e, normalized to 1.9.4.61
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CX

# **Adding power function and primality test**

In [ ]:
%%writefile power_computation.cpp
#include <boost/multiprecision/cpp_int.hpp>
#include <benchmark/benchmark.h>
#include <iostream>
#include <boost/random.hpp>
#include <random>  // for std::random_device
using namespace boost::multiprecision;
using namespace boost::multiprecision::literals;

// Fixed-size unsigned integer types for 512-bit and 1024-bit arithmetic
using uint512 = number<cpp_int_backend<512, 512, unsigned_magnitude, unchecked, void>>;
using uint1024 = number<cpp_int_backend<1024, 1024, unsigned_magnitude, unchecked, void>>;

/** Set to false if x^2 mod n = 1 with x != 1, n-1 */
bool isProbablyPrime;

/**
 * Computes a^p mod n using fast exponentiation and checks during
 * computation whether there is an x with x^2 mod n = 1 and x != 1, n-1
 *
 * @param a The base
 * @param p The exponent
 * @param n The modulus
 * @return (a^p) mod n
 */

uint512 power(uint512 a, uint512 p, uint512 n){
  if (p == 0) return 1;
  uint512 x = power(a, p/2, n);

  /* square the result, use uint1024 to avoid overflow */
  uint512 result = static_cast<uint512>(uint1024(x) * uint1024(x) % n);

  /* check whether x^2 mod n = 1 and x != 1, n-1 */
  if (result == 1 && x != 1 && x != n - 1)
  {
    isProbablyPrime = false;
  }

  /* if exponent is odd, multiply by base once more */
  if(p % 2 == 1)
  {
    // to avoid overflow
    result = static_cast<uint512>(uint1024(result) * uint1024(a) % n);
  }
  return result;
}


/**
 * Generates a random 512-bit unsigned integer
 *
 * @return A random 512-bit unsigned integer
 */
uint512 rand512(){
    // Use std::random_device to get a seed - hardware random device
    std::random_device rd;
    unsigned int seed = rd();

    // Seed Boost Mersenne Twister
    boost::random::mt19937 gen(seed);

    // Create independent_bits_engine for 512-bit numbers
    boost::random::independent_bits_engine<boost::random::mt19937, 512, uint512> bit_engine(gen);

    // Generate a random 512-bit number
    uint512 r = bit_engine();
    return r;
}


/**
 * Executes the randomized primality test for a chosen at random
 *
 * @param n The number to test for primality
 * @return true if n is probably prime, false if composite
 */
bool primality_test(uint512 n)
{
    /* handle edge cases */
    if (n < 2) return false;
    if (n == 2 || n == 3) return true;
    if (n % 2 == 0) return false;

    isProbablyPrime = true;

    /* choose random base a in range [2, n-2] */
    uint512 a = 2 + (rand512() % (n - 3));
    uint512 result = power(a, n-1, n);
    if(result != 1 || !isProbablyPrime)
      return false;
    else
      return true;
}

bool primality_test_multiple(uint512 n, int k = 10) {
    for (int i = 0; i < k; i++) {
        if (!primality_test(n)) {
            return false;
        }
    }
    return true;
}


static void BM_primality_test_range(benchmark::State& state) {
    /* get bit size from benchmark parameter */
    int bits = state.range(0);

    /* create test number: 2^bits - 1 */
    uint512 n = (uint512(1) << bits) - 1;

    for (auto _ : state) {
        bool result = primality_test(n);
        /* force compiler to keep result and not optimize away computation */
        /* ensure result is used to prevent dead code elimination */
        benchmark::DoNotOptimize(result);
    }

    /* set complexity parameter for Big-O analysis */
    state.SetComplexityN(state.range(0));
}

/**
 * Runs the probabilistic primality test once on multiple numbers and checks accuracy.
 *
 * @param test_cases Vector of pairs (number, expected_result).
 * @return A vector of numbers that failed the primality test.
 */
std::vector<uint512> run_primality_analysis(std::vector<std::pair<uint512, bool>> test_cases){
    std::vector<uint512> failed;
    int correct = 0, total = 0;
    for (size_t i = 0; i < test_cases.size(); ++i) {
        uint512 number = test_cases[i].first;
        bool expected = test_cases[i].second;
        bool result = primality_test(number);
        bool passed = (result == expected);

        std::cout << number << " -> " << " [" << (passed ? "PASS" : "FAIL") << "]\n";

        if (passed) correct++;
        else failed.push_back(number);
        total++;
    }

    std::cout << "Accuracy: " << correct << "/" << total << " ("
              << (100.0 * correct / total) << "%)\n\n\n";
    return failed;

}
/**
 * Runs the probabilistic primality test multiple times for higher reliability.
 *
 * @param test_cases Vector of pairs (number, expected_result).
 * @param iterations Number of independent test iterations per number.
 */
void run_primality_multiple_times_analysis(std::vector<std::pair<uint512, bool>> test_cases, int iterations){
    int correct = 0, total = 0;
    for (size_t i = 0; i < test_cases.size(); ++i) {
        uint512 number = test_cases[i].first;
        bool expected = test_cases[i].second;
        bool result = primality_test_multiple(number, iterations);
        bool passed = (result == expected);

        std::cout << number << " -> " << " [" << (passed ? "PASS" : "FAIL") << "]\n";

        if (passed) correct++;
        total++;
    }

    std::cout << "Accuracy: " << correct << "/" << total << " ("
              << (100.0 * correct / total) << "%)\n\n\n";
}

/*
 * Verify prime numbers with small and large prime numbers
 */
void verify_prime_numbers(){
    std::vector<std::pair<uint512, bool>> small_primes;


    small_primes.push_back(std::make_pair(2, true));
    small_primes.push_back(std::make_pair(3, true));
    small_primes.push_back(std::make_pair(5, true));
    small_primes.push_back(std::make_pair(7, true));
    small_primes.push_back(std::make_pair(11, true));
    small_primes.push_back(std::make_pair(17, true));

    std::cout << "Verifying primality test for small prime number:\n";
    std::cout << "-------------------------------------------------\n";

    run_primality_analysis(small_primes);

    // Known large primes for testing
    std::vector<std::pair<uint512, bool>> large_primes;

    uint512 n("11143019148881759237873801284515278662968125831867159375678486632106426763973191700733386455424606216934328203110917478865612050852736050307804680419342659");
    uint512 q("8043138641042387298329835447927111867872602587882568118710455122936884667146642941423316172059374682604084196590996864061369776986783015020770769334075763");

    large_primes.push_back(std::make_pair(n, true));
    large_primes.push_back(std::make_pair(q, true));


    std::cout << "Verifying primality test for large prime number:\n";
    std::cout << "-------------------------------------------------\n";

    run_primality_analysis(large_primes);


}

/**
 * Verify composite numbers, in case of inaccurate computation of critical composite numbers, rerun primality test multiple times
 * to use a newly generated random base 'a' to minimize the probability of a false prime result.
 */
void verify_composite_numbers(){
    std::cout << "Verifying primality test for composite numbers:\n";
    std::cout << "-------------------------------------------------\n";
    std::vector<std::pair<uint512, bool>> composite_numbers;
    composite_numbers.push_back(std::make_pair(0, false));
    composite_numbers.push_back(std::make_pair(1, false));
    composite_numbers.push_back(std::make_pair(4, false));
    composite_numbers.push_back(std::make_pair(12, false));
    composite_numbers.push_back(std::make_pair(1204, false));
    composite_numbers.push_back(std::make_pair(12045, false));

    // Critical Composite numbers
    composite_numbers.push_back(std::make_pair(341, false));
    composite_numbers.push_back(std::make_pair(561, false));
    composite_numbers.push_back(std::make_pair(2047, false));


    uint512 p("3317044064679887385961981");
    composite_numbers.push_back(std::make_pair(p, false));

    std::vector<uint512> failed = run_primality_analysis(composite_numbers);
    std::vector<std::pair<uint512, bool>> retrials;
    if(failed.size() > 0){
      std::cout << "Inaccurate output for composite number, run primalityTest multiple times for newly generated random base a: \n";
      for (int i = 0; i < failed.size(); i++) {
        retrials.push_back(std::make_pair(failed[i], false));
        run_primality_multiple_times_analysis(retrials, 10);
      }
    }
}

/** Configure benchmark: test 8 to 512 bits, doubling each time */
BENCHMARK(BM_primality_test_range)
    ->RangeMultiplier(2)
    ->Range(8, 512)
    ->Complexity()
    ->Unit(benchmark::kMicrosecond);

/**
 * Main function: runs tests then benchmarks
 *
 * @param argc Argument count
 * @param argv Argument values
 * @return Exit status
 */
int main(int argc, char** argv) {
    verify_prime_numbers();
    verify_composite_numbers();

    /* initialize and run benchmarks */
     ::benchmark::Initialize(&argc, argv);
    if (::benchmark::ReportUnrecognizedArguments(argc, argv)) return 1;
    ::benchmark::RunSpecifiedBenchmarks();
    return 0;
}

Overwriting power_computation.cpp


In [ ]:
!g++ power_computation.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram2

In [ ]:
!./testprogram2

Verifying primality test for small prime number:
-------------------------------------------------
2 ->  [PASS]
3 ->  [PASS]
5 ->  [PASS]
7 ->  [PASS]
11 ->  [PASS]
17 ->  [PASS]
Accuracy: 6/6 (100%)


Verifying primality test for large prime number:
-------------------------------------------------
11143019148881759237873801284515278662968125831867159375678486632106426763973191700733386455424606216934328203110917478865612050852736050307804680419342659 ->  [PASS]
8043138641042387298329835447927111867872602587882568118710455122936884667146642941423316172059374682604084196590996864061369776986783015020770769334075763 ->  [PASS]
Accuracy: 2/2 (100%)


Verifying primality test for composite numbers:
-------------------------------------------------
0 ->  [PASS]
1 ->  [PASS]
4 ->  [PASS]
12 ->  [PASS]
1204 ->  [PASS]
12045 ->  [PASS]
341 ->  [PASS]
561 ->  [PASS]
2047 ->  [PASS]
3317044064679887385961981 ->  [PASS]
Accuracy: 10/10 (100%)


2025-10-21T21:29:39+00:00
Running ./testprogram2
Ru